In [13]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint

In [9]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/61.0.3163.100 Safari/537.36'
}

city = 'los-angeles-ca'  # Need better locations, gps coords and ranges?

In [19]:
pages = 3
prices = []
with requests.Session() as session:
	# Multiple pages (find out num returned per page)
	for i in range(1, pages+1):
		url = f'https://www.zillow.com/homes/for_sale/{city}/3-_beds/2.0-_baths/{i}_p/'
		response = session.get(url, headers=req_headers)
		soup = BeautifulSoup(response.text, "html.parser")
		for j in enumerate(soup):
		    prices.extend(list(soup.find_all (class_='list-card-price')))
pprint(prices)

[<div class="list-card-price">$793,900</div>,
 <div class="list-card-price">$511,900</div>,
 <div class="list-card-price">$847,900</div>,
 <div class="list-card-price">$980,900</div>,
 <div class="list-card-price">$849,900</div>,
 <div class="list-card-price">$1,032,900</div>,
 <div class="list-card-price">$1,299,900</div>,
 <div class="list-card-price">$1,179,000</div>,
 <div class="list-card-price">$630,900</div>,
 <div class="list-card-price">$793,900</div>,
 <div class="list-card-price">$511,900</div>,
 <div class="list-card-price">$847,900</div>,
 <div class="list-card-price">$980,900</div>,
 <div class="list-card-price">$849,900</div>,
 <div class="list-card-price">$1,032,900</div>,
 <div class="list-card-price">$1,299,900</div>,
 <div class="list-card-price">$1,179,000</div>,
 <div class="list-card-price">$630,900</div>,
 <div class="list-card-price">$793,900</div>,
 <div class="list-card-price">$511,900</div>,
 <div class="list-card-price">$847,900</div>,
 <div class="list-card